행렬 벡터는 복합 데이터Compound data를 표현하는 좋은 예입니다.

여기서는 이전에 만들었던 [accumulate](http://wikibootup.github.io/sicp/6-accumulate.html)를 이용하여 벡터의 기본 연산을 구현하도록 하겠습니다.

In [1]:
from modules.basic import accumulate, accumulate_n

\* 여기서 벡터는 $v = (v_i)$으로 표현되는 일차원 행렬이고, 행렬은 $m = (m_{ij})$으로 표현되어 이차원 행렬입니다.

먼저, 내적을 구현하겠습니다.

두 벡터의 1. 각 성분을 곱한 결과를 2. 모두 더하면 그게 내적입니다. ( $\sum_iv_iw_i$ )

1.의 과정은 `map`을 이용해서,

2.의 과정은 `accumulate`를 이용하여 코드를 구현하면,

In [2]:
def dot_product(v, w):
    return accumulate(
        lambda p1, p2: p1 + p2,
        0,
        list(map(lambda m1, m2: m1 * m2, v, w)))

아래의 한 문제만 검증해보면,

$
    A =
    \begin{pmatrix}
    1\ 2\ 3\ 4
    \end{pmatrix},
    \ B =
    \begin{pmatrix}
    5\ 6\ 7\ 8
    \end{pmatrix}
    ,\ A·B = B·A =\ ?
$

In [3]:
A = [1, 2, 3, 4]
B = [5, 6, 7, 8]

In [4]:
dot_product(A, B) == dot_product(B, A)

True

In [5]:
dot_product(A, B)

70

다음으로, '행렬과 벡터의 곱'을 구현하겠습니다.

행렬의 1. 한 행의 각 성분을 벡터의 각 성분에 곱하는 과정을 2. 행렬의 처음부터 끝 행까지 반복, 3. 행 마다의 결과를 벡터로 만들면 그게 '행렬-벡터 곱'입니다. ( $\sum_jm_{ij}v_j$ )

1.과 2.의 과정은 내적`dot_product`을 구할 때와 같으므로 `dot_product`를 사용하고

3.은 `map`을 이용하여 행마다 그 과정을 반복하여 나온 결과를 행의 값으로 만들어 코드를 구현하면,

In [19]:
def matrix_by_vector(m, v):
    return list(map(
        lambda row: dot_product(row, v),
        m))

'행렬-벡터 곱'의 결과는 행렬의 각 행과 벡터를 내적한 값들로 만든 행렬과 같으므로

이것으로 아래의 경우를 검증하면,

$
    M =
    \begin{pmatrix}
    1\ 2\ 3\\
    4\ 5\ 6\\
    7\ 8\ 9\\
    \end{pmatrix},
    \ V =
    \begin{pmatrix}
    5\ 6\ 7
    \end{pmatrix}
$

In [35]:
M = [(1,2,3), (4,5,6), (7,8,9)]
V = [5,6,7]
RESULT = matrix_by_vector(M, V)

In [36]:
RESULT

[38, 92, 146]

In [37]:
def test_matrix_by_vector_using_dot_product(m, v, result):
    if m == []:
        pass
    else:
        return result[0] == dot_product(m[0], v) and \
            test_matrix_by_vector_using_dot_product(m[1:], v, result[1:])

In [38]:
test_matrix_by_vector_using_dot_product(M, V, RESULT)

하지만 행렬의 행 순서를 거꾸로 뒤집어 다른 값을 가지도록 한 후 검증해보면,

In [39]:
test_matrix_by_vector_using_dot_product(M, V, RESULT[::-1])

False

다음으로, '전치 행렬transposed matrix'을 구현하겠습니다.

같은 열에 위치한 값을 모아 행으로 바꾸어주면 끝이기 때문에

`accumulate_n` 함수를 사용하여 구현하면,

In [146]:
def transpose(mat):
    return accumulate_n(lambda e1, e2: [e2] + [e1] if type(e2) is int else e2 + [e1], [], mat[::-1])

In [147]:
M

[(1, 2, 3), (4, 5, 6), (7, 8, 9)]

In [148]:
transpose(M)

[[1, 4, 7], [2, 5, 8], [3, 6, 9]]

이제 '행렬-행렬 곱'을 구현하겠습니다.

방금전 구현한 '행렬-벡터 곱'을 이용하면 쉽게 만들 수 있습니다.

우선, 행렬을 여러 벡터가 모인 테이블이라고 보겠습니다.

그러면 1. 행렬의 각 열에(각각의 벡터) 대하여 2. '행렬-벡터 곱'을 수행한 결과들을 결합하면 '행렬-행렬 곱'이 된다는 사실을 알 수 있습니다. ( $P_{ij} = \sum_km_{ik}v_{kj}$ )

1.은 map을 이용해 각 열에 2.를 적용하고

2.는 `matrix_by_vector`를 사용하여 코드를 구현하면,